In [ ]:
import pandas as pd
import os
import json
import glob
import sqlite3

In [ ]:
#ouvert les paramètres décrits dans le fichier JSON
with open('../parameters.json') as json_data:
    params = json.load(json_data)


In [ ]:
params

In [ ]:
#utilisation des paramètres pour créer un DATA et TMP
DATA_DIR = params['DIRECTORIES']['DATA_DIR']
TMP_DIR = params['DIRECTORIES']['TMP_DIR']
PRICE_FILE = params['FILES']['PRICE_FILE']
STATION_FILE = params['FILES']['STATION_FILE']
SERVICE_FILE = params['FILES']['SERVICE_FILE']
GAS_DB = os.path.join(DATA_DIR, params['DATABASE']['GAS_DB'])

In [ ]:
gas_files = glob.glob(os.path.join(DATA_DIR,"Prix*.zip"))
gas_files


In [ ]:
stations_file = 'Stations2016.zip'
service_file = 'Services2016.zip'


In [ ]:
#ouverture de la table
#df2014 = pd.read_csv(os.path.join(DATA_DIR, gas_files[0]), sep=";", header=None, dtype = {1 : str})
df2014 = pd.read_csv(gas_files[0], sep=";", header=None, dtype = {1 : str})
df2014.columns = ['id_stations', 
                  'code_postal',
                  'type_station', 
                  'latitude',
                  'longitude',
                  'date_releve',
                  'type_carburant', 
                 'libelle_carburant',
                  'prix']

In [ ]:
df2014.head()

In [ ]:
dfall = pd.DataFrame()

for year in gas_files:
    df = pd.read_csv(year,
                     sep=';', header=None, dtype = {1: str}
                    )
    df.columns = ['id_station', 'code_postal', 'type_station',
                  'latitude', 'longitude', 'date_releve',
                  'type_carburant', 'libelle_carburant', 'prix']
    dfall = pd.concat([dfall, df], axis=0)

assert dfall.shape[1] == 9, "Pas le bon nombre de colonnes"

In [ ]:
dfall

In [ ]:
#donne la liste des fichiers
import glob

gas_files = glob.glob(os.path.join(DATA_DIR,"Prix*.zip"))

In [ ]:
gas_files

In [ ]:
dfstations = pd.read_csv(os.path.join(DATA_DIR, stations_file),sep ="|", encoding ='utf-8', header = None, dtype = {1 : str})

dfstations.columns = ['id_station', 'code_postal', 'type_de_station', 'latitude', 'longitude', 'adresse', 'ville']

In [ ]:
dfstations.head()

In [ ]:
dfservices = pd.read_csv(os.path.join(DATA_DIR, service_file),sep ="|", encoding ='utf-8', header = None, dtype = {1 : str})

dfservices.columns = ['id_station', 'code_postal', 'type_de_station', 'latitude', 'longitude', 'services']

In [ ]:
dfservices.head()

In [ ]:
dfall.to_pickle(os.path.join(DATA_DIR, PRICE_FILE))
dfstations.to_pickle(os.path.join(DATA_DIR, STATION_FILE))
dfservices.to_pickle(os.path.join(DATA_DIR, SERVICE_FILE))

In [ ]:
# test le temps de lecture
%timeit
testdf = pd.read_pickle(os.path.join(DATA_DIR, PRICE_FILE))

In [ ]:
cnx = sqlite3.connect(GAS_DB)
dfall.to_sql('prices', cnx ,index = False, if_exists = "replace")

In [ ]:
dfservices.to_sql('Services', cnx, index = False, if_exists = "replace")
dfstations.to_sql('Stations', cnx, index = False, if_exists = "replace")

In [ ]:
pd.read_sql_query('select * from prices where code_postal ="13001"', cnx)

## Fin du Script de chargement

In [ ]:
cur = cnx.cursor()

In [ ]:
cur.execute('select * from prices where prix >1400')
res = cur.fetchall()

In [ ]:
res_df = pd.read_sql_query('select * from prices where prix >1400', cnx)

In [ ]:
import numpy as np

In [ ]:
dfall['date_releve'] = pd.to_datetime(dfall.date_releve)

In [ ]:
dfall['semaine'] = dfall.date_releve.dt.week + 52 * (dfall.date_releve.dt.year - 2014)


In [ ]:
moyenne_hebdo_df = (dfall
    .dropna(subset=['date_releve'])
    .groupby(['libelle_carburant', pd.Grouper(key = "date_releve", freq ="2W-MON" )])
    .agg({'prix': np.mean})
                   ).reset_index()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
graph = sns.FacetGrid((moyenne_hebdo_df), hue="libelle_carburant",  size =10, aspect=4/3)
graph.map(plt.plot, "date_releve","prix")

In [ ]:
graph

In [ ]:
dfservices.head()

In [ ]:
dfservices.services.str.split(",")

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [149]:
dfservices["listes_services"] = dfservices.services.str.split(",")

In [ ]:
vectorize = CountVectorizer()
vectorize.fit(df)

In [158]:
_tokenize = lambda x: x.split(',')

In [161]:
vectorize = CountVectorizer(tokenizer= _tokenize, lowercase=False)

In [162]:
vectorize.fit_transform(dfservices.services.fillna('Rien'))

<12391x25 sparse matrix of type '<class 'numpy.int64'>'
	with 75115 stored elements in Compressed Sparse Row format>

In [163]:
vectorize.vocabulary_

{'Aire de camping-cars': 0,
 'Automate CB': 1,
 'Baie de service auto': 2,
 'Bar': 3,
 'Boutique alimentaire': 4,
 'Boutique non alimentaire': 5,
 'Carburant qualité supérieure': 6,
 'Douches': 7,
 'Espace bébé / change': 8,
 'GPL': 9,
 'Lavage haute-pression': 10,
 'Lavage multi-programmes': 11,
 'Laverie': 12,
 'Location de véhicule': 13,
 'Piste poids lourds': 14,
 'Relais colis': 15,
 'Restauration sur place': 16,
 'Restauration à emporter': 17,
 'Rien': 18,
 'Station de gonflage': 19,
 'Station de lavage': 20,
 'Toilettes publiques': 21,
 'Vente de fioul domestique': 22,
 'Vente de gaz domestique': 23,
 'Vente de pétrole lampant': 24}